In [1]:
# update the lunch error next time update days_check
import pandas as pd
import numpy as np

In [59]:
#importing the division file
division = pd.read_excel('division_tt.xlsx', encoding='utf-8')
division.head()

# importing the class room file
cr = pd.read_excel('cr_tt.xlsx', encoding='utf-8')
cr.head()

# importing the teacher file
teacher = pd.read_excel('teacher_tt.xlsx', encoding='utf-8')
teacher.head()

# creating the base of data frame
empty_list = [0,'null']
rows = ['8:00 AM-9:00 AM','9:00 AM-10:00 AM','10:00 AM-11:00 AM','11:00 AM-12:00 PM','12:00 PM-1:00 PM','1:00 PM-2:00 PM','2:00 PM-3:00 PM','3:00 PM-4:00 PM','4:00 PM-5:00 PM','5:00 PM-6:00 PM']
column = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']


# getting the list of divisions
division_list = division['Division'].copy()
for i in range(len(division_list)):
    division_list[i].split('-')
    division_list[i] = division_list[i][0]
    

division_list = division_list.unique()


# getting the list of teachers
teacher_list = teacher['Teacher'].copy()
teacher_list = teacher_list.unique()



# getting the list of class rooms available
cr_list = cr['cr'].copy()
cr_list = cr_list.unique()




# creating a dummy timetable for each division
division_df = []
for i in range(len(division_list)):
    new_str = division_list[i]
    
    new_str = pd.DataFrame( index = rows, columns = column)
    new_str.iloc[0,0] = empty_list
    new_str = new_str.ffill(axis = 0)
    new_str = new_str.ffill(axis = 1)
    division_df.append(new_str)

    
# creating a dummy timetable for each teacher
teachers_df = []
for i in range(len(teacher_list)):
    new_str = teacher_list[i]
    
    new_str = pd.DataFrame( index = rows, columns = column)
    new_str.iloc[0,0] = empty_list
    new_str = new_str.ffill(axis = 0)
    new_str = new_str.ffill(axis = 1)
    teachers_df.append(new_str)   

    
# creating a dummy timetable for each class room
cr_df = []
for i in range(len(cr_list)):
    new_str = cr_list[i]
    
    new_str = pd.DataFrame( index = rows, columns = column)
    new_str.iloc[0,0] = empty_list
    new_str = new_str.ffill(axis = 0)
    new_str = new_str.ffill(axis = 1)
    cr_df.append(new_str)


In [60]:
def lunch_break(start,end,current):
    
    current = current.split('-')
    current = current[0].split(' ')
    am_pm = current[1]
    current = current[0].split(':')
    current = int(current[0])
    if(am_pm == 'PM'):
        if(current%12 != 0):
            current = current + 12

    start = '11:00 AM'        
    start = start.split(' ')
    am_pm = start[1]
    start = start[0].split(':')
    start = int(start[0])
    if(am_pm == 'PM'):
        if(start%12 != 0):
            start = start + 12

    end = '2:00 PM'        
    end = end.split(' ')
    am_pm = end[1]
    end = end[0].split(':')
    end = int(end[0])
    if(am_pm == 'PM'):
        if(end%12 != 0):
            end = end + 12

    lunch = 0
    
    for i in range(start,end):
        if(current == i):
            lunch = 1
            break
    return lunch        

In [61]:
for divs in range(len(division_list)):
    div = division_list[divs]
    df1 = division.where(division['Division'].str.contains(div))
    df1 = df1[pd.notnull(df1['Division'])]
    df1 = df1.reset_index(drop=True)
    completed = []
    for i in range(len(df1)):
        completed.append(0)
    df1['completed'] = completed
    df1.head()


    df2 = teacher.where(teacher['Divisions'].str.contains(div))
    df2 = df2[pd.notnull(df2['Divisions'])]
    df2 = df2.reset_index(drop=True)
    df2


    days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday']
    check = [0,0,0,0,0,0]
    lunch_break = [0,0,0,0,0,0]
    days_check = [list(a) for a in zip(days, check,lunch_break)]
    days_check

    flag_div_completed = 0 # flag becomes 1 when all the subjects in that division have been allocated
    i = 0 # for subject
    div_max_limit = 8
    while(flag_div_completed == 0):
        # select the subject and find out the classes where the subject can be conducted
        sub = df1['Subject'][i]
        dummy_df = cr.where(cr['Subject'] == sub)
        dummy_df = dummy_df[pd.notnull(dummy_df['Subject'])]
        if(dummy_df.empty):
            # subject is a theory / tutorial and will be condected in a class
            dummy_df = cr.where(cr['division'] == div) 
            dummy_df = dummy_df[pd.notnull(dummy_df['division'])]
            cr_div = dummy_df['cr'].iloc[0]
            cr_no_index = np.where(cr_list == cr_div)[0]
            cr_no_index = cr_no_index.astype(int)
            # the class room where the subject can be conducted
            cr_no_index = cr_no_index[0]

            #if subject hours are not completed then set lecture allocated to 0
            if(df1['completed'].iloc[i] == 0):
                lec_allocated = 0
            # running the loop for 6 days a week
            for j in range(7):

                if(lec_allocated == 1):
                    break
                for k in range(10):
                    # checking if the class room is free
                    if(cr_df[cr_no_index][days[j]][k][0] == 0):
                        # getting the index pf particular teachers tt
                        teacher_sub = df2['Teacher'].where(df2['Subject'] == sub ).dropna()
                        teacher_sub = teacher_sub.iloc[0]
                        dummy_df = teacher.where(teacher['Teacher'] == teacher_sub)
                        dummy_df = dummy_df[pd.notnull(dummy_df['Teacher'])]

                        teacher_no_index = np.where(teacher_list == teacher_sub)[0]
                        teacher_no_index = teacher_no_index.astype(int)
                        teacher_no_index = teacher_no_index[0] 

                        #checking if the teacher is free
                        if(teachers_df[teacher_no_index][days[j]][k][0] == 0):

                            # getting the index of the tt of particular division
                            div_index = np.where(division_list == div)[0]

                            div_index = div_index.astype(int)
                            div_index = div_index[0]

                            #checking if the division is free
                            if(division_df[div_index][days[j]][k][0] == 0 and days_check[j][1] == 0):
                                #getting the total no. of hours that subject is conducted in a week
                                hours = df1['hours in week'].where(df1['Subject'] == sub).dropna().iloc[0]

                                hours = int(hours)
                                flag_break = 0
                                if(hours > 2):
                                    hours = 2

                                dummy_df1 = cr_df[cr_no_index].copy()
                                dummy_df2 = teachers_df[teacher_no_index].copy()
                                dummy_df3 = division_df[div_index].copy()
                                try:
                                    for z in range(2):

                                        if(dummy_df1.iloc[k+z,j][0] == 1):
                                            flag_break = 1
                                        if(dummy_df2.iloc[k+z,j][0] == 1):
                                            flag_break = 1
                                        if(dummy_df3.iloc[k+z,j][0] == 1):
                                            flag_break = 1 
                                except:
                                    flag_break = 1

                                if(flag_break == 1):
                                    break

                                # allocating the lecture in all the time tables
                                for z in range(int(hours)):

                                    cr_df[cr_no_index].iat[k+z,j] = [1,div + sub]

                                    teachers_df[teacher_no_index].iat[k+z,j] = [1,div + sub]


                                    division_df[div_index].iat[k+z,j] = [1,div + sub]

                                #trying to allocate lunch to the division if the time is right
                                lunch = 0
                                try:
                                    current = division_df[div_index].index[k+hours]
                                    lunch = lunch_break('11:00 AM','2:00 PM',current)


                                except:
                                    pass

                                # trying to give a break to the teacher
                                try:
                                    teachers_df[teacher_no_index].iat[k+hours,j] = [1,'break']

                                    if(days_check[j][2] == 0):
                                        if(lunch == 1):
                                            if(division_df[div_index].iloc[k+hours,j][0] == 0):
                                                days_check[j][2] = 1
                                                division_df[div_index].iat[k+hours,j] = [1,'lunch break']
                                except:

                                    pass

                                #subtracting the hours from the df1
                                df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] - hours

                                # checking whetehr the maximum limit to the no. of lectures in a day has been reached or not
                                div_hour = 0
                                for z in range(len(division_df)):
                                    div_hour = div_hour + division_df[div_index].iloc[:,j][z][0]
                                if(div_hour>=div_max_limit):
                                    days_check[j][1] = 1   



                                #checkking whether the subject hours have been completed
                                #checking whether all the subjects of the division have been completed or not
                                if(df1['hours in week'].where(df1['Subject'] == sub).dropna().reset_index(drop=True)[0] == 0):
                                    df1['completed'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = 1
                                    lec_allocated = 1
                                    div_not_completed = 0
                                    for z in range(len(df1)):
                                        if(df1['completed'][z] == 0):
                                            div_not_completed = 1
                                    if(div_not_completed == 0):
                                        flag_div_completed = 1


                                    break
                                else:
                                    lec_allocated = 1
                                    break



                            else:
                                continue



                        else:
                            continue

                    else:
                        continue


        else:
            # subject is a practical and will be conducted in a particular lab
            cr_div = dummy_df['cr'].iloc[0]
            cr_no_index = np.where(cr_list == cr_div)[0]
            cr_no_index = cr_no_index.astype(int)
            cr_no_index = cr_no_index[0]
            if(df1['completed'].iloc[i] == 0):
                lec_allocated = 0
            for j in range(7):

                if(lec_allocated == 1):
                    break
                for k in range(10):

                    if(cr_df[cr_no_index][days[j]][k][0] == 0):
                        # getting the index pf particular teachers tt
                        teacher_sub = df2['Teacher'].where(df2['Subject'] == sub ).dropna()
                        teacher_sub = teacher_sub.iloc[0]
                        dummy_df = teacher.where(teacher['Teacher'] == teacher_sub)
                        dummy_df = dummy_df[pd.notnull(dummy_df['Teacher'])]

                        teacher_no_index = np.where(teacher_list == teacher_sub)[0]
                        #
                        teacher_no_index = teacher_no_index.astype(int)
                        teacher_no_index = teacher_no_index[0] 
                        if(teachers_df[teacher_no_index][days[j]][k][0] == 0):

                            # getting the index of the tt of particular division
                            div_index = np.where(division_list == div)[0]
                            #
                            div_index = div_index.astype(int)
                            div_index = div_index[0]
                            if(division_df[div_index][days[j]][k][0] == 0 and days_check[j][1] == 0):
                                #getting the total no. of hours that subject is conducted in a week
                                hours = df1['hours in week'].where(df1['Subject'] == sub).dropna().iloc[0]

                                hours = int(hours)
                                flag_break = 0
                                if(hours > 2):
                                    hours = 2

                                dummy_df1 = cr_df[cr_no_index].copy()
                                dummy_df2 = teachers_df[teacher_no_index].copy()
                                dummy_df3 = division_df[div_index].copy()
                                try:
                                    for z in range(2):


                                        if(dummy_df1.iloc[k+z,j][0] == 1):
                                            flag_break = 1
                                        if(dummy_df2.iloc[k+z,j][0] == 1):
                                            flag_break = 1
                                        if(dummy_df3.iloc[k+z,j][0] == 1):
                                            flag_break = 1 
                                except:
                                    flag_break = 1


                                if(flag_break == 1):
                                    break



                                for z in range(int(hours)):

                                    cr_df[cr_no_index].iat[k+z,j] = [1,div + sub]

                                    teachers_df[teacher_no_index].iat[k+z,j] = [1,div + sub]


                                    division_df[div_index].iat[k+z,j] = [1,div + sub]

                                lunch = 0

                                try:
                                    current = division_df[div_index].index[k+hours]
                                    lunch = lunch_break('11:00 AM','2:00 PM',current)


                                except:
                                    pass

                                try:
                                    teachers_df[teacher_no_index].iat[k+hours,j] = [1,'break']
                                    if(days_check[j][2] == 0):
                                        if(lunch == 1):

                                            if(division_df[div_index].iloc[k+hours,j][0] == 0):
                                                days_check[j][2] = 1
                                                division_df[div_index].iat[k+hours,j] = [1,'lunch break']
                                except:

                                    pass


                                df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] - hours

                                div_hour = 0

                                for z in range(len(division_df)):
                                    div_hour = div_hour + division_df[div_index].iloc[:,j][z][0]
                                if(div_hour>=div_max_limit):
                                    days_check[j][1] = 1  




                                if(df1['hours in week'].where(df1['Subject'] == sub).dropna().reset_index(drop=True)[0] == 0):
                                    df1['completed'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = 1
                                    lec_allocated = 1
                                    div_not_completed = 0
                                    for z in range(len(df1)):
                                        if(df1['completed'][z] == 0):
                                            div_not_completed = 1
                                    if(div_not_completed == 0):
                                        flag_div_completed = 1


                                    break
                                else:
                                    lec_allocated = 1
                                    break



                            else:
                                continue



                        else:
                            continue

                    else:
                        continue
        i = i+1            
        if(i>= len(df1)):
            i = 0

E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
8:00 AM-9:00 AM,"[1, DISM]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
9:00 AM-10:00 AM,"[1, DISM]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
10:00 AM-11:00 AM,"[1, break]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
11:00 AM-12:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
12:00 PM-1:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
1:00 PM-2:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
2:00 PM-3:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
3:00 PM-4:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
4:00 PM-5:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
5:00 PM-6:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"


In [67]:
df1

,Division,Subject,hours in week,Combined,completed
0,H,ISM,0.0,NaN,1
1,H,BADM,0.0,NaN,1
2,H-mech,RAC(practical),0.0,NaN,1
3,H-mech,RAC(theory)_mumbai,0.0,NaN,1
4,H-mech,RAC(theory)_shirpur,0.0,G,1
5,H,LAB,0.0,NaN,1
6,H,OB,0.0,NaN,1
7,H,OM,0.0,NaN,1
8,H-mech-res,RES(theory),0.0,G,1
9,H-mech-res,RES(tutorial),0.0,NaN,1


In [45]:
df2

,Teacher,Subject,Divisions
0,Dinesh Panchal,BADM,B D E F H
1,Mehek K,LAB,E F G H
2,Kiran Desai,MM,D E GH
3,Padmanabha Aital,OM,A H
4,Geetha Iyer,FAWCM,B C H
5,Mukund Prasad,ISM,C H
6,Nilesh Balki,RES(tutorial),G H
7,Nilesh Balki,RES(theory),G H
8,Amit Chaudhary,AE(tutorial),G H
9,Amit Chaudhary,AE(theory),G H


In [214]:
flag_div_completed = 0
div = 'A'
i = 0
div_max_limit = 7
# select the subject and find out the classes where the subject can be conducted
sub = df1['Subject'][i]
dummy_df = cr.where(cr['Subject'] == sub)
dummy_df = dummy_df[pd.notnull(dummy_df['Subject'])]
if(dummy_df.empty):
    # subject is a theory / tutorial and will be condected in a class
    dummy_df = cr.where(cr['division'] == div) 
    dummy_df = dummy_df[pd.notnull(dummy_df['division'])]
    cr_div = dummy_df['cr'].iloc[0]
    cr_no_index = np.where(cr_list == cr_div)[0]
    cr_no_index = cr_no_index.astype(int)
    # the class room where the subject can be conducted
    cr_no_index = cr_no_index[0]
    
    #if subject hours are not completed then set lecture allocated to 0
    if(df1['completed'].iloc[i] == 0):
        lec_allocated = 0
    # running the loop for 6 days a week
    for j in range(7):

        if(lec_allocated == 1):
            break
        for k in range(10):
            # checking if the class room is free
            if(cr_df[cr_no_index][days[j]][k][0] == 0):
                # getting the index pf particular teachers tt
                teacher_sub = df2['Teacher'].where(df2['Subject'] == sub ).dropna()
                teacher_sub = teacher_sub.iloc[0]
                dummy_df = teacher.where(teacher['Teacher'] == teacher_sub)
                dummy_df = dummy_df[pd.notnull(dummy_df['Teacher'])]

                teacher_no_index = np.where(teacher_list == teacher_sub)[0]
                teacher_no_index = teacher_no_index.astype(int)
                teacher_no_index = teacher_no_index[0] 
                
                #checking if the teacher is free
                if(teachers_df[teacher_no_index][days[j]][k][0] == 0):

                    # getting the index of the tt of particular division
                    div_index = np.where(division_list == div)[0]
                    
                    div_index = div_index.astype(int)
                    div_index = div_index[0]
                    
                    #checking if the division is free
                    if(division_df[div_index][days[j]][k][0] == 0 and days_check[j][1] == 0):
                        #getting the total no. of hours that subject is conducted in a week
                        hours = df1['hours in week'].where(df1['Subject'] == sub).dropna().iloc[0]

                        hours = int(hours)
                        flag_break = 0
                        if(hours > 2):
                            hours = 2
                            
                        dummy_df1 = cr_df[cr_no_index].copy()
                        dummy_df2 = teachers_df[teacher_no_index].copy()
                        dummy_df3 = division_df[div_index].copy()
                        try:
                            for i in range(2):
                                print('theory')
                                if(dummy_df1.iloc[k+i,j][0] == 1):
                                    flag_break = 1
                                if(dummy_df2.iloc[k+i,j][0] == 1):
                                    flag_break = 1
                                if(dummy_df3.iloc[k+i,j][0] == 1):
                                    flag_break = 1 
                        except:
                            flag_break = 1

                        if(flag_break == 1):
                            break

                        # allocating the lecture in all the time tables
                        for i in range(int(hours)):
                            print('add')

                            cr_df[cr_no_index].iat[k+i,j] = [1,div + sub]

                            teachers_df[teacher_no_index].iat[k+i,j] = [1,div + sub]


                            division_df[div_index].iat[k+i,j] = [1,div + sub]

                        #trying to allocate lunch to the division if the time is right
                        lunch = 0
                        try:
                            current = division_df[div_index].index[k+hours]
                            lunch = lunch_break('11:00 AM','2:00 PM',current)
                            
                        
                        except:
                            pass
                        
                        # trying to give a break to the teacher
                        try:
                            teachers_df[teacher_no_index].iat[k+hours,j] = [1,'break']
                            
                            if(days_check[j][2] == 0):
                                if(lunch == 1):
                                    if(division_df[div_index].iloc[k+hours,j][0] == 0):
                                        
                                        division_df[div_index].iat[k+hours,j] = [1,'lunch break']
                        except:
                            
                            pass

                        
                        df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] - hours

                        # checking whetehr the maximum limit to the no. of lectures in a day has been reached or not
                        div_hour = 0
                        for i in range(len(division_df)):
                            div_hour = div_hour + division_df[div_index].iloc[:,j][i][0]
                        if(div_hour>=div_max_limit):
                            days_check[j][1] = 1   
                        
                        
                        
                        #checkking whether the subject hours have been completed
                        #checking whether all the subjects of the division have been completed or not
                        if(df1['hours in week'].where(df1['Subject'] == sub).dropna().reset_index(drop=True)[0] == 0):
                            df1['completed'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = 1
                            lec_allocated = 1
                            div_not_completed = 0
                            for i in range(len(df1)):
                                if(df1['completed'][i] == 0):
                                    div_not_completed = 1
                            if(div_not_completed == 0):
                                flag_div_completed = 1

                            print(flag_div_completed)
                            break
                        else:
                            lec_allocated = 1
                            break
                        
                        

                    else:
                        continue



                else:
                    continue

            else:
                continue


else:
    # subject is a practical and will be conducted in a particular lab
    cr_div = dummy_df['cr'].iloc[0]
    cr_no_index = np.where(cr_list == cr_div)[0]
    cr_no_index = cr_no_index.astype(int)
    cr_no_index = cr_no_index[0]
    if(df1['completed'].iloc[i] == 0):
        lec_allocated = 0
    for j in range(7):

        if(lec_allocated == 1):
            break
        for k in range(10):
            # checking if the class room is free
            if(cr_df[cr_no_index][days[j]][k][0] == 0):
                # getting the index pf particular teachers tt
                teacher_sub = df2['Teacher'].where(df2['Subject'] == sub ).dropna()
                teacher_sub = teacher_sub.iloc[0]
                dummy_df = teacher.where(teacher['Teacher'] == teacher_sub)
                dummy_df = dummy_df[pd.notnull(dummy_df['Teacher'])]

                teacher_no_index = np.where(teacher_list == teacher_sub)[0]
                teacher_no_index = teacher_no_index.astype(int)
                teacher_no_index = teacher_no_index[0] 
                
                #checking if the teacher is free
                if(teachers_df[teacher_no_index][days[j]][k][0] == 0):

                    # getting the index of the tt of particular division
                    div_index = np.where(division_list == div)[0]
                    
                    div_index = div_index.astype(int)
                    div_index = div_index[0]
                    
                    #checking if the division is free
                    if(division_df[div_index][days[j]][k][0] == 0 and days_check[j][1] == 0):
                        #getting the total no. of hours that subject is conducted in a week
                        hours = df1['hours in week'].where(df1['Subject'] == sub).dropna().iloc[0]

                        hours = int(hours)
                        flag_break = 0
                        if(hours > 2):
                            hours = 2
                            
                        dummy_df1 = cr_df[cr_no_index].copy()
                        dummy_df2 = teachers_df[teacher_no_index].copy()
                        dummy_df3 = division_df[div_index].copy()
                        print('prac')
                        try:
                            for i in range(2):
                                print('prac')
                                if(dummy_df1.iloc[k+i,j][0] == 1):
                                    flag_break = 1
                                if(dummy_df2.iloc[k+i,j][0] == 1):
                                    flag_break = 1
                                if(dummy_df3.iloc[k+i,j][0] == 1):
                                    flag_break = 1 
                        except:
                            flag_break = 1

                        if(flag_break == 1):
                            break

                        # allocating the lecture in all the time tables
                        for i in range(int(hours)):

                            cr_df[cr_no_index].iat[k+i,j] = [1,div + sub]

                            teachers_df[teacher_no_index].iat[k+i,j] = [1,div + sub]


                            division_df[div_index].iat[k+i,j] = [1,div + sub]

                        #trying to allocate lunch to the division if the time is right
                        lunch = 0
                        try:
                            current = division_df[div_index].index[k+hours]
                            lunch = lunch_break('11:00 AM','2:00 PM',current)
                            
                        
                        except:
                            pass
                        
                        # trying to give a break to the teacher
                        try:
                            teachers_df[teacher_no_index].iat[k+hours,j] = [1,'break']
                            
                            if(days_check[j][2] == 0):
                                if(lunch == 1):
                                    if(division_df[div_index].iloc[k+hours,j][0] == 0):
                                        
                                        division_df[div_index].iat[k+hours,j] = [1,'lunch break']
                        except:
                            
                            pass

                        
                        df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = df1['hours in week'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] - hours

                        # checking whetehr the maximum limit to the no. of lectures in a day has been reached or not
                        div_hour = 0
                        for i in range(len(division_df)):
                            div_hour = div_hour + division_df[div_index].iloc[:,j][i][0]
                        if(div_hour>=div_max_limit):
                            days_check[j][1] = 1   
                        
                        
                        
                        #checkking whether the subject hours have been completed
                        #checking whether all the subjects of the division have been completed or not
                        if(df1['hours in week'].where(df1['Subject'] == sub).dropna().reset_index(drop=True)[0] == 0):
                            df1['completed'][df1['hours in week'].where(df1['Subject'] == sub).dropna().index[0]] = 1
                            lec_allocated = 1
                            div_not_completed = 0
                            for i in range(len(df1)):
                                if(df1['completed'][i] == 0):
                                    div_not_completed = 1
                            if(div_not_completed == 0):
                                flag_div_completed = 1

                            print(flag_div_completed)
                            break
                        else:
                            lec_allocated = 1
                            break
                        
                        

                    else:
                        continue



                else:
                    continue

            else:
                continue

                
                
               

add
0


E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [209]:
division_df[div_index]

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
8:00 AM-9:00 AM,"[1, ALAB]","[1, AAI(practical)]","[0, null]","[0, null]","[0, null]","[0, null]"
9:00 AM-10:00 AM,"[1, AAI(theory)]","[1, AAI(practical)]","[0, null]","[0, null]","[0, null]","[0, null]"
10:00 AM-11:00 AM,"[1, AAI(theory)]","[1, ADWM(theory)]","[0, null]","[0, null]","[0, null]","[0, null]"
11:00 AM-12:00 PM,"[1, AMC(theory)]","[1, ADWM(theory)]","[0, null]","[0, null]","[0, null]","[0, null]"
12:00 PM-1:00 PM,"[1, AMC(theory)]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
1:00 PM-2:00 PM,"[1, AOB]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
2:00 PM-3:00 PM,"[1, AOB]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
3:00 PM-4:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
4:00 PM-5:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
5:00 PM-6:00 PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"


In [198]:
div_not_completed = 0

for i in range(len(df1)):
    if(df1['completed'][i] == 0):
        print(i)
        div_not_completed = 1
        
if(div_not_completed == 0):
    flag_div_completed = 1

print(flag_div_completed)

0

In [207]:
df1

,Division,Subject,hours in week,Combined,completed
0,A,LAB,0.0,NaN,1
1,A-it-ai,AI(theory),1.0,B,0
2,A-it-mc,MC(theory),1.0,B,0
3,A,OB,0.0,NaN,1
4,A-it-ai,AI(practical),0.0,NaN,1
5,A-it-mc,MC(practical),2.0,NaN,0
6,A-it-dwm,DWM(theory),3.0,B,0
7,A-it-sqa,SQA(theory),3.0,B,0
8,A,FAWCM,2.0,NaN,0
9,A,OM,2.0,NaN,0


In [199]:
div_not_completed = 0

for i in range(len(df1)):
    if(df1['completed'][i] == 0):
        div_not_completed = 1
        
if(div_not_completed == 0):
    flag_div_completed = 1

print(flag_div_completed)    

1


In [185]:
div_completed = 1
for i in range(len(df1)):
    if(df1['completed'][i] == 0):
        print(i)
        div_not_completed = 0

if(div_completed == 1):
    flag_div_completed = 1

print(flag_div_completed)

True

In [87]:
def lunch_break(start,end,current):
    
    current = current.split('-')
    current = current[0].split(' ')
    am_pm = current[1]
    current = current[0].split(':')
    current = int(current[0])
    if(am_pm == 'PM'):
        if(current%12 != 0):
            current = current + 12

    start = '11:00 AM'        
    start = start.split(' ')
    am_pm = start[1]
    start = start[0].split(':')
    start = int(start[0])
    if(am_pm == 'PM'):
        if(start%12 != 0):
            start = start + 12

    end = '2:00 PM'        
    end = end.split(' ')
    am_pm = end[1]
    end = end[0].split(':')
    end = int(end[0])
    if(am_pm == 'PM'):
        if(end%12 != 0):
            end = end + 12

    lunch = 0
    
    for i in range(start,end):
        if(current == i):
            lunch = 1
            break
    return lunch        
    

In [91]:
current = '10:00 AM-1:00 PM'        
lunch_break('11:00 AM','2:00 PM',current)
      

11
14
10


0

In [81]:
current = '10:00 AM-1:00 PM'
current = current.split('-')
current = current[0].split(' ')
am_pm = current[1]
current = current[0].split(':')
current = int(current[0])
if(am_pm == 'PM'):
    if(current%12 != 0):
        current = current + 12

start = '11:00 AM'        
start = start.split(' ')
am_pm = start[1]
start = start[0].split(':')
start = int(start[0])
if(am_pm == 'PM'):
    if(start%12 != 0):
        start = start + 12

end = '2:00 PM'        
end = end.split(' ')
am_pm = end[1]
end = end[0].split(':')
end = int(end[0])
if(am_pm == 'PM'):
    if(end%12 != 0):
        end = end + 12

lunch = 0
for i in range(start,end):
    if(current == i):
        lunch = 1
        break
lunch    

0

In [50]:
div = 'B'
dummy_df = cr.where(cr['division'] == div) 
dummy_df = dummy_df[pd.notnull(dummy_df['division'])]
cr_div = dummy_df['cr'].iloc[0]

cr_no_index = np.where(cr_list == cr_div)[0]

cr_no_index = cr_no_index.astype(int)
cr_no_index = cr_no_index[0]
cr_no_index

4

In [21]:
dummy_df1 = cr_df[0].copy()
for i in range(4):
    try:
        dummy_df1.iat[8+i,0] = [2,'yolo']
    except:
        print('gg')
        break
        
        
        

gg


In [20]:
dummy_df1 = cr_df[0].copy()

In [14]:
cr_df[0].iat[7,0] = [1,'yolo']

In [22]:
cr_df[0]

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
8:00AM-9:00AM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
9:00AM-10:00AM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
10:00AM-11:00AM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
11:00AM-12:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
12:00PM-1:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
1:00PM-2:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
2:00PM-3:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
3:00PM-4:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
4:00PM-5:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
5:00PM-6:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"


In [17]:
if(df1['hours in week'].where(df1['Subject'] == 'LAB').dropna()[0] == 0):
    df1['completed'][df1['hours in week'].where(df1['Subject'] == 'LAB').dropna().index[0]] = 1
    print('Complete')
else:
    print('not complete')

not complete


In [21]:
cr_df[0]['Monday'][0]

[0, 'LAB']

In [43]:
division_df[div_index]

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday
8:00AM-9:00AM,"[1, AAI(practical)]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
9:00AM-10:00AM,"[1, AAI(practical)]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
10:00AM-11:00AM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
11:00AM-12:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
12:00PM-1:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
1:00PM-2:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
2:00PM-3:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
3:00PM-4:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
4:00PM-5:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"
5:00PM-6:00PM,"[0, null]","[0, null]","[0, null]","[0, null]","[0, null]","[0, null]"


In [53]:
df1['Subject'][4] = 'AI(practical)'

E:\softwares\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
df1['hours in week'].where(df1['Subject'] == df1['Subject'][1]).dropna().reset_index(drop=True)[0]

3.0

In [55]:
df1['hours in week'].where(df1['Subject'] == 'AI(practical)').dropna().reset_index(drop=True)[0]

2.0

In [62]:
cr_df[0][days[0]][0]

[0, 'null']

In [61]:
days[0]

'Monday'

In [69]:
sub = 'LAB'

teacher_sub = df2['Teacher'].where(df2['Subject'] == sub ).dropna()
teacher_sub = teacher_sub.iloc[0]
dummy_df = teacher.where(teacher['Teacher'] == teacher_sub)
dummy_df = dummy_df[pd.notnull(dummy_df['Teacher'])]
teacher_no_index = np.where(teacher_list == teacher_sub)[0]
teacher_no_index.astype(int)
teacher_no_index[0]   

11

In [73]:
hours = 2.0

In [74]:
type(hours)

float

In [75]:
hours = df1['hours in week'].where(df1['Subject'] == 'LAB').dropna().iloc[0]
hours

1.0

In [76]:
type(hours)

numpy.float64

In [80]:
hours = hours.astype(int)
type(hours)

numpy.int32

In [14]:
a = 0
for i in range(3):
        if(a > 1):
            break
        for j in range(3):
            a = a + j
            print(a)
            if(a > 1):
                break
                

0
1
3
